<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-3%20%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

## 랜덤포레스트

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.inspection import permutation_importance # .feature_importances_ 대체용으로 사용 추천
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

In [13]:
# wine = pd.read_csv('https://bit.ly/wine_csv_data')
# wine.head()

In [14]:
dir_name = '../data/analysis/'
company = 'hyunmotor_ml.csv'
df = pd.read_csv(dir_name+company)

AttributeError: 'DataFrame' object has no attribute 'heaed'

In [15]:
df.head()

,date,ixic,ixic_cr,ixic_f,ixic_f_cr,dji,dji_cr,dji_f,dji_f_cr,spx,...,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
0,2020-01-03,9020.77,-0.79,8810.0,-0.92,28634.88,-0.81,28602.0,-0.83,3234.85,...,-2.157890,-1.375459,-0.069882,-0.372194,-0.176484,-0.004374,-0.157971,-0.001729,0.646228,0.015258
1,2020-01-06,9071.47,0.56,8847.5,0.43,28703.38,0.24,28642.0,0.14,3246.28,...,-2.315127,-4.214006,-0.256679,0.613659,0.057901,0.000000,1.489012,-0.004103,1.628066,-0.003191
2,2020-01-07,9068.58,-0.03,8853.0,0.06,28583.68,-0.42,28526.0,-0.40,3237.18,...,-3.285418,-0.779225,-0.500663,-1.812237,0.023334,0.000864,-0.217492,0.000000,1.077663,-0.014691
3,2020-01-08,9129.24,0.67,8944.5,1.03,28745.09,0.56,28770.0,0.86,3253.05,...,-4.211582,-1.227007,-0.203398,-3.273825,0.504283,0.000401,-0.000602,-0.011434,0.915090,0.000000
4,2020-01-09,9203.43,0.81,9009.0,0.72,28956.90,0.74,28929.0,0.55,3274.70,...,-4.984682,-4.448025,-0.210094,0.015186,-0.009640,-0.007395,-0.322602,-0.002245,0.476310,0.005150


In [16]:
list1 = df.columns[:60]

Index(['date', 'ixic', 'ixic_cr', 'ixic_f', 'ixic_f_cr', 'dji', 'dji_cr',
       'dji_f', 'dji_f_cr', 'spx', 'spx_cr', 'spx_f', 'spx_f_cr', 'kospi',
       'kospi_cr', 'kosdaq', 'kosdaq_cr', 'wti', 'wti_cr', 'dxy', 'dxy_cr',
       'krw', 'krw_cr', 'bond_usa_10', 'bond_usa_10_cr', 'bond_usa_2',
       'bond_usa_2_cr', 'bond_usa_3m', 'bond_usa_3m_cr', 'bond_kor_10',
       'bond_kor_10_cr', 'bond_kor_2', 'bond_kor_2_cr', 'vix', 'vix_cr', 'sox',
       'sox_cr', 'cpi', 'cpi_anticipated', 'cpi_previous', 'gold', 'gold_cr',
       'fed_rate', 'fed_rate_fore', 'fed_rate_prev', 'fed_rate_ann',
       'fed_rate_imp', 'bok_rate', 'fu_usa_date', 'op_usa_date', 'qw_usa_day',
       'fu_kor_date', 'op_kor_date', 'dw_kor_day', 'dayofweek', 'retail',
       'retail_class', 'retail_cr', 'retail_days', 'foreigner'],
      dtype='object')

In [17]:
list2 = df.columns[60:120]

Index(['foreigner_class', 'foreigner_cr', 'foreigner_days', 'institution',
       'institution_class', 'institution_cr', 'institution_days', 'financial',
       'financial_class', 'financial_cr', 'financial_days', 'invtrust',
       'invtrust_class', 'invtrust_days', 'pension', 'pension_class',
       'pension_cr', 'pension_days', 'privequity', 'privequity_class',
       'privequity_cr', 'privequity_days', 'bank', 'bank_class', 'bank_days',
       'insurance', 'insurance_class', 'insurance_days', 'financeetc',
       'financeetc_class', 'financeetc_days', 'corporateetc',
       'corporateetc_class', 'corporateetc_cr', 'corporateetc_days',
       'foreigneretc', 'foreigneretc_class', 'foreigneretc_days', 'open',
       'high', 'low', 'close', 'close_cr', 'close_cr_class', 'vol',
       'vol_percent', 'total', 'retail_ratio', 'foreigner_ratio',
       'institution_ratio', 'financial_ratio', 'invtrust_ratio',
       'pension_ratio', 'privequity_ratio', 'bank_ratio', 'insurance_ratio',
   

In [19]:
sel_column = ['date', 'ixic_cr', 'dji_cr', 'spx_cr', 'kospi_cr', 'kosdaq_cr', 
              'wti_cr', 'dxy_cr', 'dayofweek', 'retail_cr', 'retail_days', 
              'foreigner_cr', 'foreigner_days', 'institution_cr', 'institution_days', 'close_cr_class']

In [3]:
# data =df.iloc[:, :3].values
# target = wine.iloc[:, -1]

In [24]:
data =df.loc[:, sel_column].values
target = df.loc[:, 'close_cr_class']

In [26]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [32]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1, )

In [33]:
scores = cross_validate(rf,
                        X= train_input, y=train_target,
                        return_train_score=True,  # 훈련데이터 결과도 리턴
#                         cv=3,
                        n_jobs=-1)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [34]:
scores

{'fit_time': array([0.00199866, 0.00199962, 0.00099921, 0.00200129, 0.00099659]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([nan, nan, nan, nan, nan]),
 'train_score': array([nan, nan, nan, nan, nan])}

In [35]:
print('test score : {} \ntrain score: {}'.format(scores['test_score'].mean(), scores['train_score'].mean() ))

test score : nan 
train score: nan


In [9]:
scores_v = cross_val_score(rf,
                        X= train_input, y=train_target,
                        cv=5,
                        n_jobs=-1)

In [10]:
scores_v

array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926])

cross_val_score, cross_validate의 차이는 test_score만 공통으로 갖고 있음.

In [11]:
rf.fit(train_input, train_target)
rf.score(test_input, test_target)

0.8892307692307693

In [12]:
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

OOB(out of bag): 부트스트랩 샘플에 포함되지 않는 샘플. <br> 이것을 검증세트로 이용하여 평가할 수 있음(아래에서 oob_score_True 지정)

In [13]:
rf_oob = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf_oob.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [14]:
rf_oob.oob_score_  #위의 test score와 결과가 비슷함

0.8934000384837406

## 엑스트라트리

부트스트랩 샘플 사용하지 않고 즉 결정트리를 만들때 전체 데이터를 사용 <br> 결정트리에서처럼 불순도 게산하지 않고, random으로 feature를 선정하여 무작위로 분할함. <br> 속도가 빠름 == DecisionTreeClassifier 매개변수 splitter='random'로 선정한 트리와 동일

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

In [16]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et,
                        X=train_input, y=train_target,
                        cv=5,
                        return_train_score=True, n_jobs=-1)

In [17]:
print('train score: {} \n test score: {}'.format(scores['train_score'].mean(), scores['test_score'].mean()))

train score: 0.9974503966084433 
 test score: 0.8887848893166506


In [18]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

## 그레이디언트 부스팅

로지스틱 손실함수, 평균제곱오차 손실함수의 오차를 줄이기 위하여 회귀트리를 계속 추가하여 사용. Learning rate를 조정하여 최저점을 벗어나지 않도록 진행. 최초 max_depth=3을 지정하여 정확도가 떨어지나 계속 회귀트리를 추가하여 끌어 올림.

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [20]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb,
                        X=train_input, y=train_target,
                        return_train_score=True,
                        n_jobs=-1)

In [21]:
print('train score: {} \n test score: {}'.format(scores['train_score'].mean(), scores['test_score'].mean()))

train score: 0.8881086892152563 
 test score: 0.8720430147331015


In [22]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb,
                        X=train_input, y=train_target,
                        return_train_score=True,
                        n_jobs=-1)

In [23]:
print('train score: {} \n test score: {}'.format(scores['train_score'].mean(), scores['test_score'].mean()))

train score: 0.9464595437171814 
 test score: 0.8780082549788999


In [24]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

In [25]:
gb.score(train_input, train_target)

0.9382335963055609

## 히스토그램 기반 부스팅

255구간으로 나누어 진행. 1개의 빈칸을 준비하고 있는데, 이 곳은 결측치 이상치등을 모아서 진행함. 따라서 결측치 등이 있는 자료를 그대로 사용해도 됨.

In [26]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [27]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [28]:
print('train score: {} \n test score: {}'.format(scores['train_score'].mean(), scores['test_score'].mean()))

train score: 0.9321723946453317 
 test score: 0.8801241948619236


### Permutation Importance <br>
추천하는 feature 중요도 확인 방법

In [29]:
from sklearn.inspection import permutation_importance

In [30]:
hgb = HistGradientBoostingClassifier()
hgb.fit(train_input, train_target)

HistGradientBoostingClassifier()

In [31]:
result = permutation_importance(hgb,
                                X=train_input, y=train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)

In [32]:
result

{'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
         0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
        [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
         0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
        [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
         0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]]),
 'importances_mean': array([0.08876275, 0.23438522, 0.08027708]),
 'importances_std': array([0.00382333, 0.00401363, 0.00477012])}

In [33]:
result['importances_mean']

array([0.08876275, 0.23438522, 0.08027708])

In [34]:
result_test = permutation_importance(hgb,
                                X=test_input, y=test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)

In [35]:
result['importances_mean']

array([0.08876275, 0.23438522, 0.08027708])

#### XGBoost

In [36]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42) # 'hist': HistGradientBoostingClassifier와 동일한 결과
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


#### LightGBM

In [37]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
